tar -C /tmp/ayu -zvxf xyz.tar.gz

In [6]:
import os,sys,pickle,time
import pandas as pd
import numpy as np
import subprocess
from multiprocessing import Pool 
import shutil
from tqdm import tqdm

In [2]:
root_dir = "../data_hdd4t/"
company_names = next(os.walk(root_dir))[1]
company_names

['亞太',
 '台灣碩網',
 '中嘉寬頻',
 '凱擘',
 '台灣之星',
 '遠傳',
 '台哥大',
 '台基開發',
 '中華電信',
 '台灣固網',
 '台固媒體']

In [3]:
def parallel_untar(dir_path):
    total, used, free = shutil.disk_usage(dir_path)
    if (free<total*0.05) or (free // (2**30) < 130):
        cmd = ['rmdir',dir_path]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
        out, err = p.communicate()
        print("!Error:", err, '!')
        print(dir_path)
        return dir_path
    else:
        cmd = ["tar","-C", dir_path,
           "-zvxf", dir_path.replace('_dir','.tar.gz')]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
        out, err = p.communicate()
        print("!Error:", err, '!')
    

## For ISP's Pcap
parallel by function name

In [4]:
# %%time
paths = []
days = ['2020-01-09','2020-01-10','2020-01-11'] #指定日期，清空掉就是全部日子
for name in tqdm(company_names):
    if name != '中華電信': #指定電信公司，註解掉就是全部
        continue
#     print('=== Running:',name,'===')
    pcap_path = root_dir + name + '/pcap/'
    file_names = next(os.walk(pcap_path))[2]
    file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
    new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
    for dir_name in new_dir_names:
        path = pcap_path+dir_name
        try:
            if days != []:
                for day in days:
                    if day in path:
                        os.makedirs(path, exist_ok=True)
                        paths.append(path)
            else:
                os.makedirs(path, exist_ok=True)
                paths.append(path)
        except NameError:
            os.makedirs(path, exist_ok=True)
            paths.append(path)
        
paths
# paths = []
#     break

100%|██████████| 11/11 [00:00<00:00, 38383.81it/s]


['../data_hdd4t/中華電信/pcap/snort.2020-01-09_dir',
 '../data_hdd4t/中華電信/pcap/snort.2020-01-11_dir',
 '../data_hdd4t/中華電信/pcap/snort.2020-01-10_dir']

In [7]:
%%time
load_start = time.time()
pool_result = []
pool = Pool() #processes=21
for path in paths:
    r = pool.apply_async(parallel_untar,(path,))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r)
load_end = time.time() - load_start
print("Decompress Time:", '{:02f}:{:02f}:{:02f}'.format(load_end // 3600, (load_end % 3600 // 60), load_end % 60))

!Error: b'' !
!Error: b'' !
!Error: b'' !
return: <multiprocessing.pool.ApplyResult object at 0x7f1fbabfb2d0>
return: <multiprocessing.pool.ApplyResult object at 0x7f1fbabfb390>
return: <multiprocessing.pool.ApplyResult object at 0x7f1fbabfb490>
Decompress Time: 1.000000:45.000000:15.102567
CPU times: user 5.45 s, sys: 1.95 s, total: 7.4 s
Wall time: 1h 45min 15s


### remove folder recursively

In [34]:
for name in tqdm(company_names):
#     if name == '中華電信': #指定電信公司
#         continue
    pcap_path = root_dir + name + '/pcap/'
    dir_names = next(os.walk(pcap_path))[1]
    if dir_names != []:
        for folder in dir_names:
            shutil.rmtree(pcap_path+folder+'/')
            print(pcap_path+folder+'/')

  0%|          | 0/11 [00:00<?, ?it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-04_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-09_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-02_dir/


100%|██████████| 11/11 [00:04<00:00,  2.25it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-11_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-08_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-05_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-06_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-07_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-03_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-01_dir/


## For ISPs' honeypot

In [22]:
paths = []
days = ['2020-01-09','2020-01-10','2020-01-11'] #指定日期，清空掉就是全部日子
for name in tqdm(company_names):
    if name != '中華電信': #指定電信公司，註解掉就是全部
        continue
#     print('=== Running:',name,'===')
    hp_path = root_dir + name + '/honeypot/'
    date_dir_names = next(os.walk(hp_path))[1]
    if days != []:
        temp = []
        for date in date_dir_names:
            for day in days:
                if day == date:
                    temp.append(date)
        date_dir_names = temp
    else:
        pass
    for date in date_dir_names:
        hp_date_path = hp_path + date +'/'
        hp_names_dir = next(os.walk(hp_date_path))[1]
        for name in hp_names_dir:
            file_names = next(os.walk(hp_date_path+name+'/'))[2]
            file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
            new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
            for dir_name in new_dir_names:
                path = hp_date_path+name+'/'+dir_name
                os.makedirs(path, exist_ok=True)
                paths.append(path)
        
paths

100%|██████████| 11/11 [00:00<00:00, 420.87it/s]


['../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11_dir',
 '../

In [23]:
%%time
pool_result = []
pool = Pool() #processes=21
for path in paths:
    r = pool.apply_async(parallel_untar,(path,))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r)

!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4390>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4c90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4290>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4490>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4690>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4850>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4310>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163aa90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163ab90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4610>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163a750>
return: <multiprocessing.poo

'2020-01-11'

In [49]:


total, used, free = shutil.disk_usage(paths[0])

print("Total: %d GiB" % (total // (2**30)))
print("Used: %d GiB" % (used // (2**30)))
print("Free: %d GiB" % (free // (2**30)))

Total: 3667 GiB
Used: 2367 GiB
Free: 1113 GiB


In [34]:
if 1<2 and (2<3:
    print(1)

1


In [12]:
os.makedirs(pcap_path+new_dir_names[0], exist_ok=True)

In [5]:
!ls  ../

anaconda3  data  data_hdd4t  Download  Leo_code


In [25]:
cmd = ["ls",'../']
p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
out, err = p.communicate()
print(str(out))

b'anaconda3\ndata\ndata_hdd4t\nDownload\nLeo_code\n'


b''